In [1]:
!nvidia-smi

Sat May 15 06:02:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:00:05.0 Off |                  Off |
| N/A   40C    P0    56W / 250W |   2387MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [3]:
# !pip install ipywidgets

In [4]:
# !pip install -U git+https://github.com/albumentations-team/albumentations

In [5]:
# 처음 한번만 받으면 됨
# !wget http://cs.jhu.edu/~syqiao/DetectoRS/DetectoRS_R50-0f1c8080.pth

In [6]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기

# 2x로 연결
cfg = Config.fromfile('./configs/detectors/LJY_detectors_cascade_rcnn_r50_1x_coco.py')
PREFIX = '../../input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (1333, 800)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (1333, 800)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = [(1000,600), (1333,800), (1666, 1000)]

cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]

cfg.work_dir = './work_dirs/DetectoRS_R50'
cfg.load_from = 'DetectoRS_R50-0f1c8080.pth'

# cfg.model.roi_head.bbox_head.num_classes = 11 # num_classes config에서 수정
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.runner.max_epochs = 24

cfg.log_config = dict(  # config to register logger hook
    interval=40,  # Interval to print the log
    hooks=[
        dict(type='TensorboardLoggerHook'),  # The Tensorboard logger is also supported
        dict(type='TextLoggerHook')
])

# data augmentation
cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

cfg.albu_train_transforms = [
    dict(
        type='ShiftScaleRotate',
        shift_limit=0.0625,
        scale_limit=0.0,
        rotate_limit=0,
        interpolation=1,
        p=0.5),
    dict(
        type='RandomBrightnessContrast',
        brightness_limit=[0.1, 0.3],
        contrast_limit=[0.1, 0.3],
        p=0.2),
    dict(
        type='OneOf',
        transforms=[
            dict(
                type='RGBShift',
                r_shift_limit=10,
                g_shift_limit=10,
                b_shift_limit=10,
                p=1.0),
            dict(
                type='HueSaturationValue',
                hue_shift_limit=20,
                sat_shift_limit=30,
                val_shift_limit=20,
                p=1.0)
        ],
        p=0.1),
    dict(type='JpegCompression', quality_lower=85, quality_upper=95, p=0.2),
    dict(type='ChannelShuffle', p=0.1),
    dict(
        type='OneOf',
        transforms=[
            dict(type='Blur', blur_limit=3, p=1.0),
            dict(type='MedianBlur', blur_limit=3, p=1.0)
        ],
        p=0.1),
]

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='Pad', size_divisor=32),
    dict(
        type='Albu',
        transforms=cfg.albu_train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            'gt_masks': 'masks',
            'gt_bboxes': 'bboxes'
        },
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='DefaultFormatBundle'),
    dict(
        type='Collect',
        keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks'],
        meta_keys=('filename', 'ori_shape', 'img_shape', 'img_norm_cfg',
                   'pad_shape', 'scale_factor'))
]

cfg.data.train.pipeline = cfg.train_pipeline

# TTA
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=[(1333,600),(1333,800),(1333,1000)],
        flip=True,
        flip_direction=["vertical", "horizontal"],
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
cfg.data.test.pipeline = cfg.test_pipeline

# scheduler 2x
# cfg.log_config = dict(  # config to register logger hook
#     interval=40,  # Interval to print the log
#     hooks=[
#         dict(type='TensorboardLoggerHook'),  # The Tensorboard logger is also supported
#         dict(type='TextLoggerHook')
# ])


print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='CascadeRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='DetectoRS_ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        conv_cfg=dict(type='ConvAWS'),
        sac=dict(type='SAC', use_deform=True),
        stage_with_sac=(False, True, True, True),
        output_img=True),
    neck=dict(
        type='RFP',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5,
        rfp_steps=2,
        aspp_out_channels=64,
        aspp_dilations=(1, 3, 6, 1),
        rfp_backbone=dict(
            rfp_inplanes=256,
            type='DetectoRS_ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_

In [7]:
model = build_detector(cfg.model)

2021-05-15 06:02:36,105 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-15 06:02:36,106 - mmdet - INFO - Use load_from_torchvision loader
2021-05-15 06:02:36,398 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.conv2.weight_diff, layer2.0.conv2.switch.weight, layer2.0.conv2.switch.bias, layer2.0.conv2.pre_context.weight, layer2.0.conv2.pre_context.bias, layer2.0.conv2.post_context.weight, layer2.0.conv2.post_context.bias, layer2.0.conv2.offset_s.weight, layer2.0.conv2.offset_s.bias, layer2.0.conv2.offset_l.weight, layer2.0.conv2.offset_l.bias, layer2.1.conv2.weight_diff, layer2.1.conv2.switch.weight, layer2.1.conv2.switch.bias, layer2.1.conv2.pre_context.weight, layer2.1.conv2.pre_context.bias, layer2.1.conv2.post_context.weight, layer2.1.conv2.post_context.bias, layer2.1.conv2.offset_s.weight, layer2.1.conv2.offset_s.bias, layer2.1.conv2.offset

In [8]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.91s)
creating index...
index created!


/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:870: FutureWarning: This class has been deprecated. Please use ImageCompression
  FutureWarning,


In [9]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-15 06:02:45,027 - mmdet - INFO - load checkpoint from DetectoRS_R50-0f1c8080.pth
2021-05-15 06:02:45,028 - mmdet - INFO - Use load_from_local loader


Done (t=0.89s)
creating index...
index created!


2021-05-15 06:02:45,472 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: bbox_head.0.fc_cls.weight, bbox_head.0.fc_cls.bias, bbox_head.0.fc_reg.weight, bbox_head.0.fc_reg.bias, bbox_head.0.shared_fcs.0.weight, bbox_head.0.shared_fcs.0.bias, bbox_head.0.shared_fcs.1.weight, bbox_head.0.shared_fcs.1.bias, bbox_head.1.fc_cls.weight, bbox_head.1.fc_cls.bias, bbox_head.1.fc_reg.weight, bbox_head.1.fc_reg.bias, bbox_head.1.shared_fcs.0.weight, bbox_head.1.shared_fcs.0.bias, bbox_head.1.shared_fcs.1.weight, bbox_head.1.shared_fcs.1.bias, bbox_head.2.fc_cls.weight, bbox_head.2.fc_cls.bias, bbox_head.2.fc_reg.weight, bbox_head.2.fc_reg.bias, bbox_head.2.shared_fcs.0.weight, bbox_head.2.shared_fcs.0.bias, bbox_head.2.shared_fcs.1.weight, bbox_head.2.shared_fcs.1.bias, mask_head.0.convs.0.conv.weight, mask_head.0.convs.0.conv.bias, mask_head.0.convs.1.conv.weight, mask_head.0.convs.1.conv.bias, mask_head.0.convs.2.conv.weight, mask_hea

KeyboardInterrupt: 

In [ ]:
!nvidia-smi